In [ ]:
pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 42.8 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyspark: filename=pyspark-3.5.4-py2.py3-none-any.whl size=317849827 sha256=dd896da64829c046f76fae6896ec683734ea0bc077186ad38bb116ca5b7ef967
  Stored in directory: /home/codespace/.cache/pip/wheels/13/92/64/da92a3521323cc629fdf25dd56eb26938e08014c1b57ad3759
Successfully built pyspark

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("my_app").getOrCreate()



25/02/21 17:03:21 WARN Utils: Your hostname, codespaces-7aea8b resolves to a loopback address: 127.0.0.1; using 10.0.4.3 instead (on interface eth0)
25/02/21 17:03:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/21 17:03:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/02/21 17:03:23 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [6]:
df = spark.range(500).toDF('number')
df.head()

Row(number=0)

In [5]:
df = df.select(df['number']+10).rename('number')
df.head()

AttributeError: 'DataFrame' object has no attribute 'rename'

In [11]:
spark.read.format('json').load('/workspaces/retail_d/data/flight-data/json/2015-summary.json').schema


StructType([StructField('DEST_COUNTRY_NAME', StringType(), True), StructField('ORIGIN_COUNTRY_NAME', StringType(), True), StructField('count', LongType(), True)])